## PART 6B: Camera-Projector Feedback Illumination (Example)

In [1]:
## Initialize everything

%run part6a_projector-camera-correspondence.ipynb

Importing libraries...
Done!
Initialize camera and default settings...
Done!
Define functions needed to capture images...
Done!
Generating patterns for projector...
Done!


Imported opencv 3.2.0
Imported PIL4.0.0
Capture new correspondences or load from file


new_correspondences(description=u'Capture new', style=ButtonStyle())

Output()

load_correspondences(description=u'Load from file', style=ButtonStyle())

...

project_button(description=u'Start (raw)', style=ButtonStyle())

Output()

Project color camera image back onto scene:


project_c_button(description=u'Start (color)', style=ButtonStyle())

Output()

In [2]:
# (3.a) create grey image using pixel value difference between projector pattern and captured image
# attempted to use standard deviation as a metric for success, doesn't really work
def feedback_illum_naive():
    lightcrafter.set_pattern(20 * numpy.uint16(numpy.ones((360,640))))    
    im = get_raw_image()
    patt = grey # guess
    for i in range(4):
        print('round ' + str(i))

        # project guess
        warp = cv.warpPerspective(patt.astype(numpy.uint8), M, (640,360))
        lightcrafter.set_pattern(warp/2)
        time.sleep(1)

        # get image
        im = get_raw_image()
        # show img
        print('im'); plt.imshow(im); plt.show();

        # grey - im
        diff = numpy.add(grey, -1*im);

        # use diff to come up with new guess
        scale = 2 ** i
        patt = numpy.add(patt, diff/scale)

# Button setup template
class fin_button(widgets.Button):
    output = widgets.Output()
    @output.capture()
    def on_button_clicked(b):
        clear_output()
        feedback_illum_naive()
print("Perform (naive) feedback illumination to get featureless image")
button1 = fin_button(description="Start")
button1.on_click(fin_button.on_button_clicked)
display(button1)
display(fin_button.output)

Perform (naive) feedback illumination to get featureless image


fin_button(description=u'Start', style=ButtonStyle())

Output()

In [3]:
# (3.b) feedback illumination based on assumption that recorded pixel value b by camera is b=ax+c, 
# a is the reflectivity (modifies the response recorded by camera of a value from the projector)
# x is the value projected from the projector
# c is the illumination of the rest of the scene (a baseline to consider as black)
# Treat x as our guess to get b=target

def feedback_illum():
    target = 127 # might need to adjust based on baseline black

    # get a baseline value for black
    lightcrafter.set_pattern(numpy.zeros((360,640)))
    time.sleep(1)
    base_black = numpy.average(get_raw_image())
    c = numpy.full((200,320), base_black)
    print("baseline black value=" + str(base_black))

    # start with grey projection as our 'guess'
    x = grey/2

    # do some number of iterations
    for i in range(6):
        # cast x in case its type was changed
        x = x.astype(numpy.uint8)

        # project 
        lightcrafter.set_pattern(transform_image(x, M))
        time.sleep(1)

        # image
        b = get_raw_image()
        print("round "+str(i)+": captured img");plt.imshow(b); plt.show()

        # reflectivity/modulation image, a=(b-c)/x
        # want to find y s.t. target = ay + c, y=(target-c)/a
        a = numpy.divide(numpy.add(b, -1*c), numpy.clip(x,1,255))
        y = numpy.divide(numpy.add(target, -1*c), numpy.clip(a,1,255))

        # update guess
        x = y
        
# Button setup template
class fi_button(widgets.Button):
    output = widgets.Output()
    @output.capture()
    def on_button_clicked(b):
        clear_output()
        feedback_illum()
print("Perform feedback illumination to get featureless image")
button1 = fi_button(description="Start")
button1.on_click(fi_button.on_button_clicked)
display(button1)
display(fi_button.output)

Perform feedback illumination to get featureless image


fi_button(description=u'Start', style=ButtonStyle())

Output()